In [1]:
!pip install ensemble_boxes

In [2]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [3]:
import os
os.chdir("/opt/ml/code/mmdetection_trash/ensemble")
!ls

detectors_cascade_rcnn_r50_1x.csv  ensemble.ipynb
detectors_r101_RandomRotate30.csv  faster_rcnn_baseline_pytorch.csv


In [4]:
# ensemble csv files
submission_files = ['detectors_cascade_rcnn_r50_1x.csv', 'detectors_r101_RandomRotate30.csv', 'faster_rcnn_baseline_pytorch.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [5]:
image_ids = submission_df[0]['image_id'].tolist()

In [6]:
annotation = '../../../input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [7]:
prediction_strings = []
file_names = []
iou_thr = 0.4

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

Warning. Removed 9 boxes with zero area!
Warning. Removed 81 boxes with zero area!
Warning. Removed 18 boxes with zero area!
Warning. Removed 45 boxes with zero area!
Warning. Removed 90 boxes with zero area!
Warning. Removed 99 boxes with zero area!
Warning. Removed 9 boxes with zero area!
Warning. Removed 36 boxes with zero area!
Warning. Removed 94 boxes with zero area!
Warning. Removed 9 boxes with zero area!
Warning. Removed 9 boxes with zero area!
Warning. Removed 9 boxes with zero area!
Warning. Removed 45 boxes with zero area!
Warning. Removed 72 boxes with zero area!
Warning. Removed 18 boxes with zero area!
Warning. Removed 27 boxes with zero area!
Warning. Removed 36 boxes with zero area!
Warning. Removed 27 boxes with zero area!
Warning. Removed 27 boxes with zero area!
Warning. Removed 99 boxes with zero area!
Warning. Removed 36 boxes with zero area!
Warning. Removed 27 boxes with zero area!
Warning. Removed 45 boxes with zero area!
Warning. Removed 99 boxes with zero are

In [8]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('ensemble3.csv')

submission.head()

,PredictionString,image_id
0,1 0.7318308 140.22397 51.37345 259.66284 172.4...,batch_01_vt/0021.jpg
1,1 0.7043852 0.0 241.01427 298.54163 455.08948 ...,batch_01_vt/0028.jpg
2,1 0.912573 80.30995 168.37361 367.222 377.5388...,batch_01_vt/0031.jpg
3,1 0.24037101864814758 0.0 228.05465698242188 3...,batch_01_vt/0032.jpg
4,1 0.82041365 388.36865 264.85236 487.73004 345...,batch_01_vt/0070.jpg
